##Introdução##

Os dados contêm as seguintes informações:

País - este é o país para o qual as informações de vacinação são fornecidas;
Código ISO do país - código ISO para o país;
Data - data para a entrada de dados; para algumas das datas, temos apenas as vacinações diárias, para outras, apenas o total (cumulativo);
Número total de vacinações - este é o número absoluto de imunizações totais no país;
Número total de pessoas vacinadas - uma pessoa, dependendo do esquema de imunização, receberá uma ou mais (normalmente 2) vacinas; em um determinado momento, o número de vacinações pode ser maior do que o número de pessoas;
Número total de pessoas totalmente vacinadas - este é o número de pessoas que receberam todo o conjunto de imunizações de acordo com o esquema de imunização (normalmente 2); em um determinado momento, pode haver um certo número de pessoas que receberam uma vacina e outro número (menor) de pessoas que receberam todas as vacinas no esquema;
Vacinações diárias (brutas) - para uma determinada entrada de dados, o número de vacinações para aquela data/país;
Vacinações diárias - para uma determinada entrada de dados, o número de vacinações para aquela data/país;
Total de vacinações por cem - razão (em porcentagem) entre o número de vacinações e a população total até a data no país;
Número total de pessoas vacinadas por cem - razão (em porcentagem) entre a população imunizada e a população total até a data no país;
Número total de pessoas totalmente vacinadas por cem - razão (em porcentagem) entre a população totalmente imunizada e a população total até a data no país;
Número de vacinações por dia - número de vacinações diárias para aquele dia e país;
Vacinações diárias por milhão - razão (em ppm) entre o número de vacinações e a população total para a data atual no país;
Vacinas usadas no país - número total de vacinas usadas no país (atualizado);
Nome da fonte - fonte da informação (autoridade nacional, organização internacional, organização local etc.);
Site da fonte - site da fonte da informação;

In [ ]:

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_df = pd.read_csv("country_vaccinations.csv")

In [ ]:
country_vaccine = data_df.groupby(["country", "iso_code", "vaccines"])[['total_vaccinations', 
                                                                       'total_vaccinations_per_hundred',
                                                                      'daily_vaccinations',
                                                                      'daily_vaccinations_per_million',
                                                                      'people_vaccinated',
                                                                      'people_vaccinated_per_hundred',
                                                                       'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred'
]].max().reset_index()
country_vaccine.columns = ["Country", "iso_code", "Vaccines", "Total vaccinations", "Percent", "Daily vaccinations", 
                           "Daily vaccinations per million", "People vaccinated", "People vaccinated per hundred",
                           'People fully vaccinated', 'People fully vaccinated percent']

Quais vacinas são usadas em cada país?

In [ ]:
vaccines = country_vaccine.Vaccines.unique()
for v in vaccines:
    countries = country_vaccine.loc[country_vaccine.Vaccines==v, 'Country'].values
    print(f"Vaccines: {v}: \nCountries: {list(countries)}\n")

In [ ]:
fig = px.choropleth(locations=country_vaccine['Country'], 
                    locationmode="country names",
                    color=country_vaccine['Vaccines'],
                    title="Countries using each vaccine (different colors for each vaccine)",
                    height = 800
                   )
fig.update_layout({'legend_orientation':'v'})
fig.update_layout({'legend_title':'Vaccine scheme'})
fig.show()

Qual esquema de vacinação é mais utilizado?

In [ ]:
vaccine = data_df.groupby(["vaccines"])[['total_vaccinations','total_vaccinations_per_hundred',
                                         'daily_vaccinations','daily_vaccinations_per_million']].max().reset_index()


vaccine.columns = ["Vaccines", "Total vaccinations", "Percent", "Daily vaccinations", 
                           "Daily vaccinations per million"]
def draw_trace_bar_vaccine(data, feature, title, xlab, ylab,color='Blue'):
    data = data.sort_values(feature, ascending=False)
    trace = go.Bar(
            x = data['Vaccines'],
            y = data[feature],
            marker=dict(color=color),
            text=data['Vaccines']
        )
    data = [trace]

    layout = dict(title = title,
              xaxis = dict(title = xlab, showticklabels=True, tickangle=45, 
                           zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                           showline=True, linewidth=2, linecolor='black', mirror=True,
                          tickfont=dict(
                            size=10,
                            color='black'),), 
              yaxis = dict(title = ylab, gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                          showline=True, linewidth=2, linecolor='black', mirror=True),
              plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)',
              hovermode = 'closest',
              height = 800
             )
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='draw_trace')
    
   

In [ ]:
 draw_trace_bar_vaccine(vaccine, 'Total vaccinations', 'Total per vaccine scheme', 'Vaccine', 'Vaccination total', "darkmagenta" )

Alguns países estão usando um esquema de vacinação misto (eles estão usando mais de uma vacina).

O mapeamento é o seguinte:

Moderna, Pfizer/BioNTech - EUA;
CNBG, Sinovac - China;
Oxford/AstraZeneca, Pfizer/BioNTech', 'Pfizer/BioNTech - Reino Unido;
Pfizer/BioNTech - principalmente UE;
Pfizer/BioNTech, Sinopharm - Emirados Árabes Unidos;
Sinovac - Turquia;
Covaxin, Covishield - Índia;

##Por países##
Para ver a distribuição do esquema de vacinação por países, usaremos representações de mapa de árvore.

Observamos as vacinações totais, os valores diários de vacinação, bem como o total de pessoas vacinadas.

Observação: clique em um item de mapa de árvore para navegar pela estrutura da árvore e expandir o ramo atual.

In [ ]:
fig = px.treemap(country_vaccine, path = ['Vaccines', 'Country'], values = 'Total vaccinations',
                title="Total vaccinations per country, grouped by vaccine scheme")
fig.show()

##Quantos são vacinados (total e como porcentagem da população)?##
Vamos olhar agora para as estatísticas dos países, independentemente do esquema de vacinação. Vamos olhar para o topo dos países por:

Número total de vacinações;
Porcentagem de vacinações de toda a população;
Número diário de vacinações;
Número diário de vacinação por milhão de população;
Pessoas vacinadas;
Porcentagem de pessoas vacinadas de toda a população.

In [ ]:
def draw_trace_bar(data, feature, title, xlab, ylab,color='Blue'):
    data = data.sort_values(feature, ascending=False)
    trace = go.Bar(
            x = data['Country'],
            y = data[feature],
            marker=dict(color=color),
            text=data['Country']
        )
    data = [trace]

    layout = dict(title = title,
              xaxis = dict(title = xlab, showticklabels=True, tickangle=45, 
                           zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                           showline=True, linewidth=2, linecolor='black', mirror=True,
                          tickfont=dict(
                            size=10,
                            color='black'),), 
              yaxis = dict(title = ylab, gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                          showline=True, linewidth=2, linecolor='black', mirror=True),
              plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)',
              hovermode = 'closest'
             )
    fig = dict(data = data, layout = layout)
    iplot(fig, filename='draw_trace')

In [ ]:
draw_trace_bar(country_vaccine, 'Total vaccinations', 'Vaccination total per country', 'Country', 'Vaccination total', "Darkgreen" )

In [ ]:
draw_trace_bar(country_vaccine, 'Percent', 'Vaccination percent per country', 'Country', 'Vaccination percent' )

In [ ]:
country_vaccine_time = data_df[["country", "vaccines", "date", 'total_vaccinations', 
                                'total_vaccinations_per_hundred',  'people_vaccinated','people_vaccinated_per_hundred',
                               'daily_vaccinations','daily_vaccinations_per_million', 
                                'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred'
                               ]].dropna()
country_vaccine_time.columns = ["Country", "Vaccines", "Date", 'Total vaccinations', 'Percent', 'People vaccinated', 'People percent',
                               "Daily vaccinations", "Daily vaccinations per million", 
                                'People fully vaccinated', 'People fully vaccinated percent']

In [ ]:
countries = ['Austria', 'Belgium', 'Bulgaria','Croatia', 'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany',
             'Greece', 'Hungary', 'Ireland', 'Israel', 'Italy', 'Latvia','Lithuania', 'Luxembourg', 'Malta',
             'Netherlands', 'Norway','Poland', 'Portugal', 'Romania', 'Serbia', 'Slovakia', 'Spain', 'Sweden',
             'United Kingdom', 'United States', 'China']

In [ ]:
def plot_time_variation_countries_group(data_df, feature, title, countries):
    data = []
    for country in countries:
        df = data_df.loc[data_df.Country==country]
        trace = go.Scatter(
            x = df['Date'],y = df[feature],
            name=country,
            mode = "markers+lines",
            marker_line_width = 1,
            marker_size = 8,
            marker_symbol = 'circle',
            text=df['Country'])
        data.append(trace)
    layout = dict(title = title,
          xaxis = dict(title = 'Date', showticklabels=True,zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                       showline=True, linewidth=2, linecolor='black', mirror=True,
                       tickfont=dict(size=10,color='darkblue'),), 
          yaxis = dict(title = feature, gridcolor='lightgrey', zeroline=True, zerolinewidth=1, zerolinecolor='grey',
                       showline=True, linewidth=2, linecolor='black', mirror=True, type="log"),
                       plot_bgcolor = 'rgba(0, 0, 0, 0)', paper_bgcolor = 'rgba(0, 0, 0, 0)',
         hovermode = 'x', 
         height=800
         )
    fig = dict(data=data, layout=layout)
    iplot(fig, filename='all_countries')

In [ ]:
plot_time_variation_countries_group(country_vaccine_time, 'Percent', 'Total vaccination percent evolution (selected countries, log scale)', countries)

In [ ]:
plot_time_variation_countries_group(country_vaccine_time, 'People fully vaccinated', 'People fully vaccinated evolution (selected countries, log scale)', countries)